In [24]:
import pandas as pd
from pathlib import Path
import AppHelpers as ah

In [15]:
jan_data_path = Path("Data/JC-202001-citibike-tripdata.csv")
feb_data_path = Path("Data/JC-202002-citibike-tripdata.csv")
mar_data_path = Path("Data/JC-202003-citibike-tripdata.csv")
apr_data_path = Path("Data/JC-202004-citibike-tripdata.csv")
may_data_path = Path("Data/JC-202005-citibike-tripdata.csv")

jan_data = pd.read_csv(jan_data_path)
feb_data = pd.read_csv(feb_data_path)
mar_data = pd.read_csv(mar_data_path)
apr_data = pd.read_csv(apr_data_path)
may_data = pd.read_csv(may_data_path)

combined_feb_may = pd.read_csv(Path("Data/combined_feb_may.csv"))

# Combine all data into a single DataFrame
combined_data = pd.concat([feb_data, mar_data, apr_data, may_data], ignore_index=True)

# add month column as first column and name it based on the month in the starttime column
combined_data.insert(0, "Month", combined_data['starttime'].str[5:7])
# convert month column value to full month name
combined_data['Month'] = combined_data['Month'].replace(['02', '03', '04', '05'], ['Feb', 'Mar', 'Apr', 'May'])

# add city column as second column and name it based on the start station latitude and start station longitude columns
# combined_data.insert(1, "City", combined_data['start station latitude'].map(str) + ", " + combined_data['start station longitude'].map(str))

# add zip code column as third column and name it based on the start station latitude and start station longitude columns
# combined_data.insert(2, "ZipCode", combined_data['start station latitude'].map(str) + ", " + combined_data['start station longitude'].map(str))

# add city column as second column and add zip column as third column
combined_data.insert(1, "City","")
combined_data.insert(2, "ZipCode","")

# convert city column value to get city name only
#combined_data['City'] = combined_data['City'].replace(['40.71958611647166, -74.04311746358871', '40.71958611647166, -74.04311746358871', '40.71958611647166, -74.04311746358871', '40.71958611647166, -74.04311746358871'], ['Jersey City', 'Jersey City', 'Jersey City', 'Jersey City'])

# combined_data.head()

In [ ]:
# slice the dataframe to get the first 100 rows
# combined_data = combined_data.iloc[0:10]
# print(len(combined_data))

# write a loop to add city name and zip code to the dataframe
for index, row in combined_data.iterrows():
    cityzip = ah.get_city_zip(row["start station latitude"], row["start station longitude"])
    combined_data.loc[index, "City"] = cityzip[0]
    combined_data.loc[index, "ZipCode"] = cityzip[1]

combined_data.head()

In [ ]:
# load the combined feb-may  data into a csv file
combined_feb_may = pd.read_csv(Path("combined_feb_may.csv"))
   
# get data where city length is greater than 0 and zipcode length is equal to 0
missingzip = combined_feb_may[(combined_feb_may['City'].str.len() > 0) & (combined_feb_may['ZipCode'].isnull())]
# get data where ZipCode as float64 column is empty
# missingzip = missingzip['ZipCode'].isnull()
missingzip.head(None)


In [46]:
cz = ah.get_city_zip(40.72534, -74.067622)
cz

('Jersey City', '')

In [25]:
# create subsets of csv data
numOfLines = 50
divby = 1000
jan_subset = pd.read_csv(jan_data_path, nrows=(len(jan_data)//divby))
feb_subset = pd.read_csv(feb_data_path, nrows=(len(feb_data)//divby))
mar_subset = pd.read_csv(mar_data_path, nrows=(len(mar_data)//divby))
apr_subset = pd.read_csv(apr_data_path, nrows=(len(apr_data)//divby))
may_subset = pd.read_csv(may_data_path, nrows=(len(mar_data)//divby))


In [15]:
# # insert Month column
# jan_subset.insert(0, "Month", "Jan")
# feb_subset.insert(0, "Month", "Feb")
# mar_subset.insert(0, "Month", "Mar")
# apr_subset.insert(0, "Month", "Apr")
# may_subset.insert(0, "Month", "May")

In [ ]:
# create a dataframe of all the subsets and add Month as first column
frames = [jan_subset, feb_subset, mar_subset, apr_subset, may_subset]
df = pd.concat(frames)
# add month column as first column and name it based on the month in the starttime column
df.insert(0, "Month", df['starttime'].str[5:7])
# convert month column value to full month name
df['Month'] = df['Month'].replace(['01', '02', '03', '04', '05'], ['Jan', 'Feb', 'Mar', 'Apr', 'May'])

# df.to_csv('combined_subset.csv', index=False)
df.head(None)


In [40]:
# create a dataframe that groupby Month and return the size of each month
df_groupby = combined_data.groupby('Month').size()
# name size column to Total Trips
df_groupby = df_groupby.rename('Total Trips')
# reset index to make Month a column
df_groupby = df_groupby.reset_index()
# save to csv
df_groupby.to_csv('total_trips.csv', index=False)


In [39]:
# create a dataframe grouped by usertype and month
df_groupby_usertype = combined_data.groupby(['Month', 'usertype']).size()
# name size column to Trips
df_groupby_usertype = df_groupby_usertype.rename('Trips')
# reset index to make Month and usertype a column
df_groupby_usertype = df_groupby_usertype.reset_index()
# save to csv
df_groupby_usertype.to_csv('trips_by_usertype.csv', index=False)


In [38]:
# get start stations
df_startstations = combined_data.groupby(['start station name']).size()
# name size column to Trips
df_startstations = df_startstations.rename('Trips')
# reset index to make start station name a column
df_startstations = df_startstations.reset_index()
# save to csv
df_startstations.to_csv('start_stations.csv', index=False)


In [17]:
# # create combined csv file
# # combined_data = combined_data.append(feb_subset, ignore_index=True)
# # combined_data = combined_data.append(mar_subset, ignore_index=True)
# # combined_data = combined_data.append(apr_subset, ignore_index=True)

# dfs = []
# dfs.append(feb_subset)
# dfs.append(mar_subset)
# dfs.append(apr_subset)
# dfs.append(may_subset)

# df_combined = pd.concat(dfs, ignore_index=True)
# df_combined.to_csv("combined_data.csv", index=False)

# # create subsets from each csv file
# jan_subset.to_csv("jan_subset.csv")
# feb_subset.to_csv("feb_subset.csv")
# mar_subset.to_csv("mar_subset.csv")
# apr_subset.to_csv("apr_subset.csv")
# may_subset.to_csv("may_subset.csv")

In [18]:
# # create groupby start station name with number of rows for each station
# grp = apr_data.groupby("start station name").size()

# # df_trips = grp.map("{:,}".format).to_frame("Trips")
# grp.to_csv("apr_startstations.csv")
# # df_trips = grp.to_frame("Trips").sort_values(["Trips"], ascending=False)
# # df_trips.head(None)

In [19]:
# # jan_data.sort_values("start station name", inplace=False)
# # janfilter = jan_data["start station name"]=="Astor Place"
# # janfilter.count()
# # jancount = jan_data[(jan_data["start station name"]=="Astor Place")]
# # groupby start station name with number of rows for each station
# jan_grouped = jan_data.groupby("start station name").count()
# jan_grouped = jan_grouped.sort_values("tripduration", ascending=False).reset_index()
# # include only station name and number of rows
# jan_grouped = jan_grouped[["start station name", "tripduration"]]
# jan_grouped.head(10)


In [20]:
# # get number of unique stations
# start_ids = jan_data.groupby(["start station name"]).count()
# end_ids = jan_data.groupby(["end station name"])
# print(len(start_ids))
# print(len(end_ids))
# start_ids.head()
